In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import PIL.Image as Image
import pathlib
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
# Get directory path to Training dataset
train_dir = pathlib.Path('../input/gender-recognition-dataset/Gender Recognition Dataset/Training')
# Get a list of all images in the Training dataset
train_image_paths = list(train_dir.glob(r'**/*.jpg'))

# Get directory path to Validation dataset
valid_dir = pathlib.Path('../input/gender-recognition-dataset/Gender Recognition Dataset/Validation')
# Get a list of all images in the Validation dataset
valid_image_paths = list(valid_dir.glob(r'**/*.jpg'))

## Image Processing

In [3]:
# Create a function to extract the labels from image filepath
def image_processing(filepath):
    labels = [str(filepath[i]).split('/')[-2]
             for i in range(len(filepath))]
    
    # Create a DataFrame and input the filepath and labels
    filepath = pd.Series(filepath, name = 'Filepath').astype(str)
    labels = pd.Series(labels, name = 'Label')
    
    df = pd.concat([filepath, labels], axis='columns')
    
    return df

In [4]:
# Create a train and validation DataFrame
train_df = image_processing(train_image_paths)
val_df = image_processing(valid_image_paths)

In [5]:
# Create DataFrame with just one label for each label
df_unique = train_df.copy().drop_duplicates(subset=['Label']).reset_index()

In [6]:
# Generate new images from dataset
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input
)

In [7]:
# Generate images using 'train_df' DataFrame
train_images = train_generator.flow_from_dataframe(
    dataframe  = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 0,
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

Found 47406 validated image filenames belonging to 2 classes.


In [8]:
# Generate images using 'val_df' DataFrame
val_images = train_generator.flow_from_dataframe(
    dataframe  = val_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 0,
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

Found 11831 validated image filenames belonging to 2 classes.


In [9]:
# Use Tensorflow pretrained model
pretrained_model = tf.keras.applications.MobileNetV2(
input_shape= (224, 224, 3),
include_top = False,
weights = 'imagenet',
pooling = 'avg'
)

# Freeze weights
pretrained_model.trainable = False

2022-09-24 12:23:36.030001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-24 12:23:36.126352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-24 12:23:36.127150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-24 12:23:36.128731: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

9420800/9406464 [==============================] - 0s 0us/step


In [10]:
# Create weights
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation = 'relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)

outputs = tf.keras.layers.Dense(2, activation = 'softmax')(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [11]:
# Train model
history = model.fit(
    train_images,
    validation_data = val_images,
    batch_size = 32,
    epochs = 20,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = 2,
            restore_best_weights = True
        )  
    ]
)

2022-09-24 12:23:40.291306: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-09-24 12:23:43.832430: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1482/1482 [==============================] - 341s 224ms/step - loss: 0.2887 - accuracy: 0.8812 - val_loss: 0.2348 - val_accuracy: 0.9059
Epoch 2/20
1482/1482 [==============================] - 129s 87ms/step - loss: 0.2310 - accuracy: 0.9065 - val_loss: 0.2195 - val_accuracy: 0.9162
Epoch 3/20
1482/1482 [==============================] - 124s 83ms/step - loss: 0.2113 - accuracy: 0.9171 - val_loss: 0.2063 - val_accuracy: 0.9173
Epoch 4/20
1482/1482 [==============================] - 125s 84ms/step - loss: 0.1989 - accuracy: 0.9205 - val_loss: 0.2007 - val_accuracy: 0.9178
Epoch 5/20
1482/1482 [==============================] - 123s 83ms/step - loss: 0.1888 - accuracy: 0.9249 - val_loss: 0.1926 - val_accuracy: 0.9236
Epoch 6/20
1482/1482 [==============================] - 127s 86ms/step - loss: 0.1795 - accuracy: 0.9308 - val_loss: 0.1940 - val_accuracy: 0.9259
Epoch 7/20
1482/1482 [==============================] - 128s 86ms/step - loss: 0.1683 - accuracy: 0.9348 - val_loss: 0.2031 - va

In [12]:
# Create labels dictionary
labels = {0: 'female',
         1: 'male'}

In [15]:
# Create a function for image processing and prediction
def output(imagepath):
    img = image.load_img(imagepath, target_size=(224, 224, 3))
    img = img_to_array(img)
    img = img/255
    img = np.expand_dims(img, [0])
    
    prediction = model.predict(img)[0]
    
    idx = prediction.argmax()
    prediction_label = labels[idx]
    
    return prediction_label

In [16]:
# Predict gender
img = output('../input/gender-classification-dataset/Validation/female/112953.jpg.jpg')
img

'female'

In [17]:
# Save model
model.save('GR.h5')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
